In [1]:
import pandas as pd
import pandas_profiling
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
from sqlalchemy import create_engine,MetaData,Table,select,engine
import pandas_profiling
import datetime

In [2]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [3]:
%matplotlib inline

In [42]:
DB_PROD={
        'drivername':   'oracle+cx_oracle',       
        'servername':   '10.100.3.76',    
        'port': '1521',    
        'sidname':'DMPRD',    
        'username': 'ODI_UNICOMER', 
        'password': 'Unicomer$01'
        }
engine_test=create_engine(DB_PROD['drivername'] + '://' + DB_PROD['username']+':' + DB_PROD['password'] + '@' + DB_PROD['servername'] + ':' + DB_PROD['port'] + '/' + DB_PROD['sidname'])

In [18]:
fuzz.ratio("Catherine M Gitau","Catherine Gitau")
fuzz.partial_ratio("Catherine M. Gitau","Catherine Gitau")
fuzz.ratio("Catherine M Gitau","Gitau Catherine")
fuzz.partial_ratio("Catherine M. Gitau","Gitau Catherine")
fuzz.token_sort_ratio("Catherine Gitau M.", "Gitau Catherine")

94

In [5]:
qryMarcaSrv="""SELECT DISTINCT MARCA_RI_NORM MARCA_SERVICES FROM
   (SELECT /*+MATERIALIZE*/
      MO.COUNTRY_CODE,
      MO.MODELO_ARTICULO_ID ID_SRV,
      MO.MODELO_ARTICULO MODELO_SRV,
      REGEXP_REPLACE(UPPER(MO.MODELO_ARTICULO),'\W+','') MODELO_SRV_NORM,
      MO.DESC_ARTICULO DESCRIPCION_SRV,
      REGEXP_REPLACE(UPPER(MO.DESC_ARTICULO),'\W+','') DESCRIPCION_SRV_NORM,
      MA.DESC_MARCA_ARTICULO MARCA_SRV,
      REGEXP_REPLACE(UPPER(MA.DESC_MARCA_ARTICULO),'\W+','') MARCA_SRV_NORM,      
      TRIM(
          REGEXP_REPLACE(
            UPPER(REGEXP_REPLACE(MA.DESC_MARCA_ARTICULO,' *\(.*$','')),
            '\W+'
          )
      ) MARCA_RI_NORM
    FROM ODS.TLR_MODELOS_ARTICULOS MO
      LEFT JOIN ODS.TLR_MARCAS_ARTICULOS MA ON MO.MARCA_ARTICULO_ID = MA.MARCA_ARTICULO_ID)
      ORDER BY 1 ASC
"""
qryMarcaRI="SELECT DISTINCT BRAND_DESC MARCA FROM DIM.DW_DIM_PRODUCT_BRAND ORDER BY MARCA ASC"

In [43]:
qryMarcaSrv="""WITH
  MARCAS_HM AS
    (
    SELECT /*+MATERIALIZE*/
      A.MARCA_RI, A.MARCA_ART
    FROM ODS.TLR_PRODUCTOS_HM A
    GROUP BY A.MARCA_RI, A.MARCA_ART
    ORDER BY A.MARCA_RI, A.MARCA_ART
    ),
  ARTICULOS_SRV AS
    (
    SELECT /*+MATERIALIZE*/
      MO.COUNTRY_CODE,
      MO.MODELO_ARTICULO_ID ID_SRV,
      MO.MODELO_ARTICULO MODELO_SRV,
      REGEXP_REPLACE(UPPER(MO.MODELO_ARTICULO),'\W+','') MODELO_SRV_NORM,
      MO.DESC_ARTICULO DESCRIPCION_SRV,
      REGEXP_REPLACE(UPPER(MO.DESC_ARTICULO),'\W+','') DESCRIPCION_SRV_NORM,
      MA.DESC_MARCA_ARTICULO MARCA_SRV,
      REGEXP_REPLACE(UPPER(MA.DESC_MARCA_ARTICULO),'\W+','') MARCA_SERVICES_NORM,
      HM.MARCA_RI,
      TRIM(
          REGEXP_REPLACE(
            UPPER(
                REGEXP_REPLACE(
                    DECODE(
                        NVL(HM.MARCA_RI,'N/A'),
                        'N/A',
                        NULL,
                        NVL(HM.MARCA_RI,'N/A')
                    ),
                    ' *\(.*$',''
                )
            ),
            '\W+'
          )
      ) MARCA_HOM_MANUAL
    FROM ODS.TLR_MODELOS_ARTICULOS MO
      LEFT JOIN ODS.TLR_MARCAS_ARTICULOS MA
        ON MO.MARCA_ARTICULO_ID = MA.MARCA_ARTICULO_ID
      LEFT JOIN MARCAS_HM HM
        ON MA.DESC_MARCA_ARTICULO = HM.MARCA_ART    
    ) SELECT * FROM ARTICULOS_SRV WHERE MARCA_SERVICES_NORM IS NOT NULL
"""

In [44]:
qryMarcaRI="SELECT DISTINCT BRAND_DESC MARCA FROM DIM.DW_DIM_PRODUCT_BRAND ORDER BY MARCA ASC"

In [45]:
names_array=[]
ratio_array=[]
def match_names(wrong_names,correct_names):
    for row in wrong_names:
        x=process.extractOne(row,correct_names,scorer=fuzz.ratio)    
        names_array.append(x[0])
        ratio_array.append(x[1])
    return names_array,ratio_array

In [21]:
dfMarcaSrv.head(20)

,country_code,id_srv,modelo_srv,modelo_srv_norm,descripcion_srv,descripcion_srv_norm,marca_srv,marca_services_norm,marca_ri,marca_hom_manual
0,503,52064,ES1512C2TL,ES1512C2TL,LAPTOP,LAPTOP,ACER,ACER,ACER (07329),ACER
1,503,52023,ES1411COFO,ES1411COFO,LAPTOP,LAPTOP,ACER,ACER,ACER (07329),ACER
2,503,62257,N15P2,N15P2,COMPUTADORA,COMPUTADORA,ACER,ACER,ACER (07329),ACER
3,503,51023,S1002145A,S1002145A,MINI LAPTOP,MINILAPTOP,ACER,ACER,ACER (07329),ACER
4,503,58364,ES47334K2ES,ES47334K2ES,LAPTOP,LAPTOP,ACER,ACER,ACER (07329),ACER
5,503,58359,E547334K2ES,E547334K2ES,LAPTOP,LAPTOP,ACER,ACER,ACER (07329),ACER
6,503,51442,S10022145A/N15P2,S10022145AN15P2,MINILAPTO,MINILAPTO,ACER,ACER,ACER (07329),ACER
7,503,58173,AZ1621RC13ES,AZ1621RC13ES,COMPUTADORA DE ESCRITORIO,COMPUTADORADEESCRITORIO,ACER,ACER,ACER (07329),ACER
8,503,53563,B1-770,B1770,TABLET,TABLET,ACER,ACER,ACER (07329),ACER
9,503,54924,E547334K2,E547334K2,LAPTOP,LAPTOP,ACER,ACER,ACER (07329),ACER


In [46]:
#Wrong marca dataset
dfMarcaSrv=pd.read_sql_query(qryMarcaSrv,engine_test)
wrong_names=dfMarcaSrv["marca_services_norm"].dropna().values

In [31]:
wrong_names

array(['ACER', 'ACER', 'ACER', ..., 'EPIC', 'EPIC', 'VIEWSAT'], dtype=object)

In [47]:
#Correct Marca dataset
dfMarcaRI=pd.read_sql_query(qryMarcaRI,engine_test)
correct_names=dfMarcaRI["marca"].values

In [48]:
name_match,ratio_match=match_names(wrong_names,correct_names)

In [49]:
dfMarcaSrv["correct_marca_name"]=pd.Series(name_match)
dfMarcaSrv["marca_name_ratio"]=pd.Series(ratio_match)

In [50]:
dfMarcaSrv.to_csv("String_Matched_Marcas_Name.csv")

In [51]:
print(dfMarcaSrv[['marca_services_norm','correct_marca_name','marca_name_ratio']].head(10))

  marca_services_norm correct_marca_name  marca_name_ratio
0                ACER               ACER               100
1                ACER               ACER               100
2                ACER               ACER               100
3                ACER               ACER               100
4                ACER               ACER               100
5                ACER               ACER               100
6                ACER               ACER               100
7                ACER               ACER               100
8                ACER               ACER               100
9                ACER               ACER               100
